# TRAINING FACE MASK DATASET

## IMPORTING NECESSARY LIBRARIES

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from keras.utils import img_to_array
from keras.utils import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import keras
from tensorflow.python.keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf



### Mounting Drive & Importing DataSet

In [5]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
directory="C:/Users/smahamka/Desktop/Intro to dl/data"
categories=["with_mask","without_mask"]


In [7]:
print("[INFO] loading images...")

data = []
labels = []
for imagePath in categories:
    path=os.path.join(directory,imagePath)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(imagePath)



[INFO] loading images...


C:\Users\smahamka\Anaconda3\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


### PERFORMING ONE HOT ENCODING

In [8]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)




### PARTIONING THE DATA INTO TRAINING AND TESTING SPLITS

In [9]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)


aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")



In [10]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=96, kernel_size=(11, 11), 
                        strides=(4, 4), activation="relu", 
                        input_shape=(224, 224, 3)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(pool_size=(3, 3), strides= (2, 2)))
model.add(keras.layers.Conv2D(filters=256, kernel_size=(5, 5), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(keras.layers.Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(4096, activation="relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation="softmax"))
model.compile(loss='binary_crossentropy', 
              optimizer=  tf.optimizers.SGD(lr=0.001), 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 54, 54, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 26, 26, 256)      1024      
 hNormalization)                                                 
                                                        

C:\Users\smahamka\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [23]:
import visualkeras
visualkeras.layered_view(model)

ModuleNotFoundError: No module named 'visualkeras'

In [14]:
print("[INFO] training head...")
# H = model.fit(
#     aug.flow(trainX, trainY, batch_size=BS),
#     steps_per_epoch=len(trainX) // BS,
#     validation_data=(testX, testY),
#     validation_steps=len(testX) // BS,
#     epochs=EPOCHS)
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    
    epochs=20)
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)



[INFO] training head...
Epoch 1/20
188/188 [==============================] - 415s 2s/step - loss: 0.6646 - accuracy: 0.7749 - val_loss: 0.5549 - val_accuracy: 0.8564
Epoch 2/20
188/188 [==============================] - 416s 2s/step - loss: 0.4530 - accuracy: 0.8486 - val_loss: 0.3081 - val_accuracy: 0.8769
Epoch 3/20
188/188 [==============================] - 412s 2s/step - loss: 0.3802 - accuracy: 0.8676 - val_loss: 0.2068 - val_accuracy: 0.9212
Epoch 4/20
188/188 [==============================] - 418s 2s/step - loss: 0.3201 - accuracy: 0.8899 - val_loss: 0.1747 - val_accuracy: 0.9418
Epoch 5/20
188/188 [==============================] - 407s 2s/step - loss: 0.2914 - accuracy: 0.8985 - val_loss: 0.1709 - val_accuracy: 0.9398
Epoch 6/20
188/188 [==============================] - 407s 2s/step - loss: 0.2595 - accuracy: 0.9106 - val_loss: 0.1370 - val_accuracy: 0.9510
Epoch 7/20
188/188 [==============================] - 399s 2s/step - loss: 0.2347 - accuracy: 0.9193 - val_loss: 0.142

### CLASSIFICATION REPORT

In [30]:
print(predIdxs)
predIdxs = np.argmax(predIdxs)
print(predIdxs)
print(testY)

# print(classification_report(testY, predIdxs))


0
0
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [31]:
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    
    epochs=20)
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

Epoch 1/20
188/188 [==============================] - 703s 4s/step - loss: 0.1211 - accuracy: 0.9611 - val_loss: 0.1175 - val_accuracy: 0.9610
Epoch 2/20
188/188 [==============================] - 682s 4s/step - loss: 0.1203 - accuracy: 0.9594 - val_loss: 0.0760 - val_accuracy: 0.9729
Epoch 3/20
188/188 [==============================] - 688s 4s/step - loss: 0.1060 - accuracy: 0.9622 - val_loss: 0.0805 - val_accuracy: 0.9649
Epoch 4/20
188/188 [==============================] - 687s 4s/step - loss: 0.1202 - accuracy: 0.9569 - val_loss: 0.0865 - val_accuracy: 0.9682
Epoch 5/20
188/188 [==============================] - 631s 3s/step - loss: 0.1162 - accuracy: 0.9599 - val_loss: 0.0745 - val_accuracy: 0.9682
Epoch 6/20
188/188 [==============================] - 530s 3s/step - loss: 0.0994 - accuracy: 0.9647 - val_loss: 0.0626 - val_accuracy: 0.9788
Epoch 7/20
188/188 [==============================] - 491s 3s/step - loss: 0.1007 - accuracy: 0.9641 - val_loss: 0.0708 - val_accuracy: 0.9729